<a href="https://colab.research.google.com/github/praveenreddi/LSTM_AMAZON_FOOD_REVIEWS/blob/master/LSTM_AMAZON_FOOD_REVIEWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8" --header="Accept-Language: en-GB,en-US;q=0.9,en;q=0.8" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-datasets/18/2157/database.sqlite.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1546748808&Signature=sUBmjAzczxO0G9nSN8pp6X%2FNRmyAqSKQK8PG%2BYTGmPVsTjcH4buImFjq1r%2FS4DpJToeTL%2F%2FejOY1UHaORWV7icpCIdjIlKSAGZyUS4gwozw6LvGCYPplWxyQwYKjcODkMnurWrTTAFj5FiTP1Er7acvAgE%2BcMbLuFvCxDoVsc76hjbAJm8kl%2B%2BJc0I3ZS4%2F4kaBaZnMPy0PUhCl9mw9%2FmDAVwsNmBVNQXT5JOTKdISgKUQLA7roDvOejvEx%2BsHpRLIJSGjQtGCh7UShk%2BJ%2BtS%2FAgctkLxP4KzDUY%2B5mFSzzjqJzPc7vPBp7bp2TclQzMff8kIwcsvmGykVTH6KkTdA%3D%3D" -O "database.sqlite.zip" -c


--2019-01-05 11:13:01--  https://storage.googleapis.com/kaggle-datasets/18/2157/database.sqlite.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1546748808&Signature=sUBmjAzczxO0G9nSN8pp6X%2FNRmyAqSKQK8PG%2BYTGmPVsTjcH4buImFjq1r%2FS4DpJToeTL%2F%2FejOY1UHaORWV7icpCIdjIlKSAGZyUS4gwozw6LvGCYPplWxyQwYKjcODkMnurWrTTAFj5FiTP1Er7acvAgE%2BcMbLuFvCxDoVsc76hjbAJm8kl%2B%2BJc0I3ZS4%2F4kaBaZnMPy0PUhCl9mw9%2FmDAVwsNmBVNQXT5JOTKdISgKUQLA7roDvOejvEx%2BsHpRLIJSGjQtGCh7UShk%2BJ%2BtS%2FAgctkLxP4KzDUY%2B5mFSzzjqJzPc7vPBp7bp2TclQzMff8kIwcsvmGykVTH6KkTdA%3D%3D
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c09::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 416 Requested range not satisfiable

    The file is already fully retrieved; nothing to do.



In [0]:
!unzip database.sqlite.zip


Archive:  database.sqlite.zip
replace database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: database.sqlite         


In [0]:


import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns


con = sqlite3.connect('database.sqlite')#Using SQLite table to read the data

#Filtering the data such that reviews will be only positive and negative and not considering review with score=3

data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 

# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = data['Score']
positiveNegative = actualScore.map(partition) 
data['Score'] = positiveNegative
print(data.shape)
data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
data.shape

(525814, 10)

In [0]:
#Sorting data according to ProductId in ascending order
sorted_data=data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [0]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [0]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(data['Id'].size*1.0)*100

69.25890143662969

In [0]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [0]:
final['Score'].size

364171

In [0]:
def polarity(X):
    if X=='positive':
        return 1
    else:
        return 0
final["Score"]=final["Score"].map(polarity)
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,1,939340800,EVERY book is educational,this witty little book makes my son laugh at l...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,1,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc..."
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,1,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,1,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,1,1018396800,A great way to learn the months,This is a book of poetry about the months of t...


In [0]:
final = final[final['ProductId'] != '2841233731']
final = final[final['ProductId'] != '0006641040']
final.shape

(364136, 10)

In [0]:
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
{'don', 'i', 'same', 'this', 'll', 'again', 'can', 'for', 'both', 'as', "don't", 'been', 'down', 'just', 'are', 'these', 'needn', "that'll", "shan't", 'his', 'than', 'while', 'about', "haven't", 'is', 'off', 'yours', "shouldn't", 'have', "didn't", 'me', 'couldn', 'ourselves', 'how', 'not', 'haven', 'the', "couldn't", 'd', "she's", 'aren', 'your', 'when', 'weren', 'shouldn', 'to', 'do', 'ain', 'ma', 'during', 'whom', 'with', 'each', 'hasn', 'but', 'who', 'does', "you'll", 'above', 'was', 'no', 'hadn', "wasn't", 'few', 'before', 'doing', "it's", 'so', 'hers', 'an', 'am', 'wasn', 'that', 'should', 'all', 'were', 'being', 'himself', 'between', 'itself', 'from', 'why', 'under', 'theirs', 'what', 'or', 'isn', 'there', "you're", "weren't", 'did', 'yourselves', "won't", 'it', 'its', "aren't", 'will', "should've", 'has', 'a', 'in', 'such', 'nor', "you'd", 've', 'which', 'after'

In [0]:
%%time

from time import time
i=0
str1=' '
final_string=[]
# all_positive_words=[] # store words from +ve reviews here
# all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
#                     if (final['Score'].values)[i] == 'positive': 
#                         all_positive_words.append(s) #list of all words used to describe positive reviews
#                     if(final['Score'].values)[i] == 'negative':
#                         all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

CPU times: user 5min 53s, sys: 51.6 ms, total: 5min 53s
Wall time: 5min 53s


In [0]:
#adding a column of CleanedText which displays the data after pre-processing of the review
final['CleanedText']=final_string  
final['CleanedText']=final['CleanedText'].str.decode("utf-8")
#below the processed review can be seen in the CleanedText Column 
print('Shape of final',final.shape)
final.head()

Shape of final (364136, 11)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
476617,515426,141278509X,AB1A5EGHHVA9M,CHelmic,1,1,1,1332547200,The best drink mix,This product by Archer Farms is the best drink...,product archer farm best drink mix ever mix fl...
22621,24751,2734888454,A1C298ITT645B6,Hugh G. Pritchard,0,0,1,1195948800,Dog Lover Delites,Our dogs just love them. I saw them in a pet ...,dog love saw pet store tag attach regard made ...
22620,24750,2734888454,A13ISQV0U9GZIC,Sandikaye,1,1,0,1192060800,made in china,My dogs loves this chicken but its a product f...,dog love chicken product china wont buy anymor...
157850,171161,7310172001,AFXMWPNS1BLU4,H. Sandler,0,0,1,1229385600,Excellent treats,I have been feeding my greyhounds these treats...,feed greyhound treat year hound littl finicki ...
157849,171160,7310172001,A74C7IARQEM1R,stucker,0,0,1,1230076800,Sophie's Treats,This is one product that my welsh terrier can ...,one product welsh terrier eat sophi food alerg...


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

##Sorting data according to Time in ascending order for Time Based Splitting
time_sorted_data = final.sort_values('Time', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

x = time_sorted_data['CleanedText'].values
y = time_sorted_data['Score']

# Finding all words in the vocabulary
count_vect = CountVectorizer() 
count_vect.fit(x)

vocabulary = count_vect.get_feature_names()
print('No. of words in the Vocabulary : ',len(vocabulary))



No. of words in the Vocabulary :  71611


In [0]:
# Storing all words in the dictionary (words as keys and index as values)
corpus = dict()
ind = 0
for sent in x:
  for word in sent.split():
    corpus.setdefault(word,[])
    corpus[word].append(ind)
    ind += 1

# Getting frequency for each word of vocabulary and storing it in a list
freq = []
for w in vocabulary:
  freq.append(len(corpus[w]))

In [0]:
# Getting Index for each word in the vocabulary
# Sorting frequencies in decreasing order
inc_index =np.argsort(np.array(freq))[::-1]

# Allocating ranks to words of vocabulary in decreasing order of frequency and storing words in a dictionary
word_rank = dict()
rank = 1
for i in inc_index:
  word_rank[vocabulary[i]] = rank
  rank +=1

In [0]:
# Converting full data into imdb format
data = []
for sent in x:
  row = []
  for word in sent.split():
    if(len(word)>1):
      row.append(word_rank[word])
  data.append(row)

# Splitting the data into 50-50 train_data and test_data
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(data, y, test_size=0.5)

In [0]:
print("No. of datapoints in X_train :",len(X_train))
print("No. of datapoints in X_test :",len(X_test))
print("Shape of Y_train :",Y_train.shape)
print("Shape of Y_test :",Y_test.shape)

No. of datapoints in X_train : 182068
No. of datapoints in X_test : 182068
Shape of Y_train : (182068,)
Shape of Y_test : (182068,)


In [0]:
print(X_train[1])

[8, 5, 76, 199, 10, 2319, 24, 671, 17, 109, 279, 74, 427]


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
# fix random seed for reproducibility
np.random.seed(7)

Using TensorFlow backend.


In [0]:
# truncate and/or pad input sequences
max_review_length = 100
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

print(X_train.shape)
print(X_train[1])

(182068, 100)
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    8    5   76  199   10 2319   24  671   17  109  279
   74  427]


In [0]:
# create the model
embedding_vector_length = 32
model_1 = Sequential()
model_1.add(Embedding(len(vocabulary)+1, embedding_vector_length, input_length=max_review_length))
model_1.add(LSTM(100))
model_1.add(Dense(1, activation='sigmoid'))
model_1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_1.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 32)           2291584   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 2,344,885
Trainable params: 2,344,885
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
 Lstm_1=model_1.fit(X_train, Y_train, epochs=5, batch_size=128)
 #Final evaluation of the model
scores = model_1.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/5
182068/182068 [==============================] - 709s 4ms/step - loss: 0.2258 - acc: 0.9117
Epoch 2/5
182068/182068 [==============================] - 708s 4ms/step - loss: 0.1738 - acc: 0.9318
Epoch 3/5
182068/182068 [==============================] - 702s 4ms/step - loss: 0.1522 - acc: 0.9406
Epoch 4/5
182068/182068 [==============================] - 725s 4ms/step - loss: 0.1326 - acc: 0.9488
Epoch 5/5
182068/182068 [==============================] - 727s 4ms/step - loss: 0.1158 - acc: 0.9555
Accuracy: 92.17%


In [0]:
# create the model
embedding_vector_length = 32
model_2 = Sequential()
model_2.add(Embedding(len(vocabulary)+1,embedding_vector_length, input_length=max_review_length))
model_2.add(LSTM(100,return_sequences=True))
model_2.add(LSTM(100))
model_2.add(Dense(1, activation='sigmoid'))
model_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model_2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 32)           2291584   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 100)          53200     
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 2,425,285
Trainable params: 2,425,285
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
 Lstm_2=model_2.fit(X_train, Y_train, epochs=5, batch_size=128)
 #Final evaluation of the model
scores = model_2.evaluate(X_test, Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/5
182068/182068 [==============================] - 1505s 8ms/step - loss: 0.2245 - acc: 0.9119
Epoch 2/5
182068/182068 [==============================] - 1108s 6ms/step - loss: 0.1719 - acc: 0.9330
Epoch 3/5
182068/182068 [==============================] - 786s 4ms/step - loss: 0.1508 - acc: 0.9417
Epoch 4/5
182068/182068 [==============================] - 784s 4ms/step - loss: 0.1295 - acc: 0.9505
Epoch 5/5
182068/182068 [==============================] - 786s 4ms/step - loss: 0.1110 - acc: 0.9584
Accuracy: 92.22%
